In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_cone(axis, angle, num_points=10):
    """
    Create a cone with its axis along the given vector.
    The cone apex is at the origin, and the cone opens with the specified angle.
    """
    # Normalize the axis vector
    axis = axis / np.linalg.norm(axis)
    
    # Create a basis where the z-axis is aligned with our cone axis
    z_new = axis
    x_new = np.array([1, 0, 0]) if np.abs(np.dot(axis, [0, 0, 1])) > 0.9 else np.array([0, 0, 1])
    x_new = x_new - np.dot(x_new, z_new) * z_new  # Make perpendicular to z_new
    x_new = x_new / np.linalg.norm(x_new)
    y_new = np.cross(z_new, x_new)  # Right-hand rule
    
    # Generate points around the cone
    theta = np.linspace(0, 2*np.pi, num_points)
    cone_radius = np.sin(angle)
    
    # Generate points at various heights
    heights = np.linspace(0, 1, 10)
    cone_points = []
    
    for h in heights:
        radius = h * cone_radius
        x_circle = radius * np.cos(theta)
        y_circle = radius * np.sin(theta)
        z_circle = h * np.cos(angle) * np.ones_like(theta)
        
        # Transform points to align with our axis
        for i in range(len(theta)):
            point = x_circle[i] * x_new + y_circle[i] * y_new + z_circle[i] * z_new
            cone_points.append(point)
            
    return np.array(cone_points)

def plot_precession(angle=30, B0=1.0, time=0.0):
    """
    Plot the precession of a spin in a magnetic field.
    
    Parameters:
    angle : float
        Angle between the spin and the z-axis in degrees
    B0 : float
        Strength of the magnetic field in Tesla
    time : float
        Time in seconds
    """
    # Clear previous plot
    clear_output(wait=True)
    
    # Create new figure
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Constants
    gyromagnetic_ratio = 42.58  # MHz/T for proton
    
    # Convert angle to radians
    angle_rad = np.radians(angle)
    
    # Calculate Larmor frequency
    omega = gyromagnetic_ratio * 1e6 * B0  # rad/s
    
    # Energy difference in frequency units (Hz)
    energy_diff = omega / (2 * np.pi)
    
    # Energy difference in eV
    energy_diff_eV = energy_diff * 6.626e-34 * 6.242e18  # neV (h*f*conversion)
    
    # Plot axes and labels
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_zlim(-1.5, 1.5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Spin Precession in Magnetic Field\n' + 
                f'Larmor Frequency: {energy_diff:.1f} MHz\n' +
                f'Energy Diff (α/β): {energy_diff_eV:.3g} neV')
    
    # Draw B0 field vector
    ax.quiver(0, 0, 0, 0, 0, 1.2, color='blue', arrow_length_ratio=0.1, label='B₀')
    
    # Current precession position (the vector we're building the cone around)
    current_angle = omega * time
    x_position = np.sin(angle_rad) * np.cos(current_angle)
    y_position = np.sin(angle_rad) * np.sin(current_angle)
    z_position = np.cos(angle_rad)
    
    moment_vector = np.array([x_position, y_position, z_position])
    
    # Draw magnetic moment vector at current position
    ax.quiver(0, 0, 0, x_position, y_position, z_position, 
             color='red', arrow_length_ratio=0.1, label='μ')
    
    # Draw the cone symmetric about the magnetic moment vector
    # We'll use a small half-angle for the cone (30 degrees)
    cone_half_angle = np.radians(30)
    cone_points = create_cone(moment_vector, cone_half_angle)
    
    # Plot the cone as a collection of points
    x_cone = cone_points[:, 0]
    y_cone = cone_points[:, 1] 
    z_cone = cone_points[:, 2]
    
    # Plot the cone surface using triangulation
    ax.scatter(x_cone, y_cone, z_cone, color='green', alpha=0.3, s=1)
    
    # Draw the circular path for precession
    theta_path = np.linspace(0, 2*np.pi, 100)
    x_path = np.sin(angle_rad) * np.cos(theta_path)
    y_path = np.sin(angle_rad) * np.sin(theta_path)
    z_path = np.cos(angle_rad) * np.ones_like(theta_path)
    ax.plot(x_path, y_path, z_path, 'b--', alpha=0.5, label='Precession path')
    
    # Current position on the precession path
    ax.scatter([x_position], [y_position], [z_position], color='red', s=50)
    
    # Add formula display
    formula_text = (
        f"$\\langle I_x \\rangle = \\frac{{\\hbar}}{{2}}\\sin({angle}°)\\cos({energy_diff:.1f}t)$\n"
        f"$\\langle I_y \\rangle = -\\frac{{\\hbar}}{{2}}\\sin({angle}°)\\sin({energy_diff:.1f}t)$"
    )
    plt.figtext(0.1, 0.05, formula_text)
    
    # Add legend
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Use widgets.interact to create the interactive visualization
widgets.interact(
    plot_precession,
    angle=(0, 180, 5),      # min, max, step
    B0=(0.1, 5.0, 0.1),     # min, max, step
    time=(0, 0.1, 0.001)     # min, max, step
);

interactive(children=(IntSlider(value=30, description='angle', max=180, step=5), FloatSlider(value=1.0, descri…

In [7]:
widgets.interact()

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import cm

def plot_precession(angle=30, B0=1.0, time=0.0):
    """
    Plot the precession of a spin in a magnetic field using atomic units.
    
    Parameters:
    angle : float
        Angle between the spin and the z-axis in degrees
    B0 : float
        Strength of the magnetic field (relative scale)
    time : float
        Time in arbitrary units (0-1 represents a full cycle)
    """
    # Clear previous plot
    clear_output(wait=True)
    
    # Create new figure
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Convert angle to radians
    angle_rad = np.radians(angle)
    
    # For simplicity, we'll use a scaled frequency that makes visualization easier
    # This makes one full time unit (0-1) correspond to one full cycle
    omega = 2 * np.pi * B0  # full cycle when time=1
    
    # Energy difference (proportional to B0)
    energy_diff = B0
    
    # Plot axes and labels
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_zlim(-1.2, 1.2)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Spin Precession in Magnetic Field\nB₀ = {B0:.1f}, Angle = {angle}°')
    
    # Draw B0 field vector
    ax.quiver(0, 0, 0, 0, 0, 1.0, color='blue', arrow_length_ratio=0.1, label='B₀')
    
    # Current precession position 
    current_angle = omega * time
    x_position = np.sin(angle_rad) * np.cos(current_angle)
    y_position = np.sin(angle_rad) * np.sin(current_angle)
    z_position = np.cos(angle_rad)
    
    # Draw magnetic moment vector
    ax.quiver(0, 0, 0, x_position, y_position, z_position, 
             color='red', arrow_length_ratio=0.1, label='μ')
    
    # Draw the cone as a surface
    # Create cone around the current position
    cone_half_angle = np.radians(20)  # Smaller cone for clarity
    
    # Generate cone using meshgrid for better surface plotting
    u = np.linspace(0, 1, 8)  # Height along cone axis (fewer points for speed)
    v = np.linspace(0, 2*np.pi, 12)  # Angle around cone (fewer points for speed)
    
    U, V = np.meshgrid(u, v)
    
    # Create cone aligned with z-axis
    X = U * np.sin(cone_half_angle) * np.cos(V)
    Y = U * np.sin(cone_half_angle) * np.sin(V)
    Z = U * np.cos(cone_half_angle)
    
    # Rotation matrix to align cone with the magnetic moment
    # First, find the rotation that takes [0,0,1] to our moment direction
    from_vec = np.array([0, 0, 1])
    to_vec = np.array([x_position, y_position, z_position])
    
    # Simple rotation for this case - just rotate around the origin
    # This simplified approach works for our visualization purposes
    # Get the current position angle in the XY plane
    phi = np.arctan2(y_position, x_position)
    
    # First rotate around y-axis to get the right zenith angle
    theta = np.arccos(z_position)
    
    # Apply rotations to each point
    X_rot, Y_rot, Z_rot = np.zeros_like(X), np.zeros_like(Y), np.zeros_like(Z)
    
    # Apply the rotations
    # First around y-axis (theta)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Rotate point
            x, y, z = X[i,j], Y[i,j], Z[i,j]
            
            # Rotate around y-axis by theta
            x_new = x * np.cos(theta) + z * np.sin(theta)
            y_new = y
            z_new = -x * np.sin(theta) + z * np.cos(theta)
            
            # Rotate around z-axis by phi
            X_rot[i,j] = x_new * np.cos(phi) - y_new * np.sin(phi)
            Y_rot[i,j] = x_new * np.sin(phi) + y_new * np.cos(phi)
            Z_rot[i,j] = z_new
    
    # Plot the cone as a surface
    ax.plot_surface(X_rot, Y_rot, Z_rot, alpha=0.3, color='green', 
                   rstride=1, cstride=1, linewidth=0)
    
    # Draw the circular path for precession (just the circle, not individual points)
    theta_path = np.linspace(0, 2*np.pi, 40)  # Fewer points for speed
    x_path = np.sin(angle_rad) * np.cos(theta_path)
    y_path = np.sin(angle_rad) * np.sin(theta_path)
    z_path = np.cos(angle_rad) * np.ones_like(theta_path)
    ax.plot(x_path, y_path, z_path, 'b--', alpha=0.5, label='Precession path')
    
    # Current position on the precession path
    ax.scatter([x_position], [y_position], [z_position], color='red', s=50)
    
    # Add simplified formula display
    formula_text = (
        f"$\\langle I_x \\rangle = \\frac{{\\hbar}}{{2}}\\sin({angle}°)\\cos(\\omega t)$\n"
        f"$\\langle I_y \\rangle = -\\frac{{\\hbar}}{{2}}\\sin({angle}°)\\sin(\\omega t)$\n"
        f"$\\omega = \\gamma B_0 = {B0:.1f}$ (in relative units)"
    )
    plt.figtext(0.1, 0.05, formula_text)
    
    # Add legend
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Use widgets.interact to create the interactive visualization
widgets.interact(
    plot_precession,
    angle=(0, 180, 15),     # Increased step size for faster updates
    B0=(0.2, 2.0, 0.2),     # Smaller range with larger steps for speed
    time=(0, 1.0, 0.05)     # Larger step size for time
);

interactive(children=(IntSlider(value=30, description='angle', max=180, step=15), FloatSlider(value=1.0, descr…

In [ ]:
#okay this is actually perfect.
# We can leave it as it is now.